In [24]:
import requests
from tqdm import tqdm
import shutil
from collections import Counter, defaultdict

headers = {"User-Agent": "My User Agent"}

In [8]:
url = input()

In [18]:
response = requests.get(url, stream=True, headers=headers)
with open('example.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

In [19]:
with open("data/titlepart2href.json", encoding="utf-8") as f:
    titlepart2href = json.load(f)

In [25]:
hrefs = Counter([href for v in titlepart2href.values() for href in v])

In [29]:
hrefs.most_common(3)

[('/53101-missing', 2),
 ('/19510-romantic-doctor-teacher-kim', 2),
 ('/38301-romantic-doctor-teacher-kim-2', 2)]

In [27]:
Counter([href[0] for href in hrefs])

Counter({'/': 19461, 'h': 15})

In [28]:
optional_prefix = "https://mydramalist.com"
len([href for href in hrefs if href.startswith(optional_prefix)])

15